In [1]:
import boto3
import pandas as pd
import numpy as np
import psycopg2
import configparser

## Leemos archivo de configuración


In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

## Nos identificamos con AWS


In [3]:
aws_rds_connection = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                              aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                              region_name='us-east-2')

## Verificamos instancias de AWS disponibles en el usuario


In [4]:
rds_instances_ids = []

aws_response = aws_rds_connection.describe_db_instances()

for response in aws_response['DBInstances']:
    rds_instances_ids.append(response['DBInstanceIdentifier'])

print(f"Instancias Disponibles: {rds_instances_ids}")

Instancias Disponibles: ['tienda-transactional']


## Creamos instancia de base de datos en AWS - PostGres


In [5]:
try:
    response = aws_rds_connection.create_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        DBName=config.get('TRANSACC', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        MasterUsername=config.get('TRANSACC', 'DB_USER'),
        MasterUserPassword=config.get('TRANSACC', 'DB_PASSWORD'),
        Port=int(config.get('TRANSACC', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=15
    )
    print(response)
except aws_rds_connection.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia ya existe")
except Exception as ex:
    print("Error!", ex)

La instancia ya existe


## Obtenemos el hostname de la instancia


In [6]:
try:
    instance = aws_rds_connection.describe_db_instances(DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print("Error!!", ex)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


## Nos conectamos a la base de datos desde Python


In [7]:
import sql_queries
sql_queries.DDL_QUERY

'\nCREATE TABLE IF NOT EXISTS categoria(\n    idcategoria INT PRIMARY KEY,\n    nombre VARCHAR(50),\n    descripcion VARCHAR(255),\n    estado BIT\n);\n\nCREATE TABLE IF NOT EXISTS articulo(\n    idarticulo INT PRIMARY KEY,\n    idcategoria INT,\n    codigo VARCHAR(50),\n    nombre VARCHAR(100),\n    precio_venta DECIMAL(11, 2),\n    stock INT,\n    descripcion VARCHAR(255),\n    imagen VARCHAR(20),\n    estado BIT,\n    FOREIGN KEY (idcategoria) REFERENCES categoria(idcategoria)\n);\n\nCREATE TABLE IF NOT EXISTS persona(\n    idpersona INT PRIMARY KEY,\n    tipo_persona VARCHAR(20),\n    nombre VARCHAR(100),\n    tipo_documento VARCHAR(20),\n    num_documento VARCHAR(20),\n    direccion VARCHAR(70),\n    telefono VARCHAR(20),\n    email VARCHAR(50)\n);\n\nCREATE TABLE IF NOT EXISTS rol(\n    idrol INT PRIMARY KEY,\n    nombre VARCHAR(30),\n    descripcion VARCHAR(255),\n    estado BIT\n);\n\nCREATE TABLE IF NOT EXISTS usuario(\n    idusuario INT PRIMARY KEY,\n    idrol INT,\n    nombr

In [8]:
try:
    db_pg_conn = psycopg2.connect(
        database=config.get('TRANSACC', 'DB_NAME'),
        user=config.get('TRANSACC', 'DB_USER'),
        password=config.get('TRANSACC', 'DB_PASSWORD'),
        host=RDS_HOSTNAME,
        port=int(config.get('TRANSACC', 'DB_PORT'))
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_pg_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("Error!", ex)

Base de Datos Creada Exitosamente


## Insertamos datos en la base de datos

In [9]:
def insertData2SQL(data_dict, table_name, driver):

    df_data = pd.DataFrame.from_records(data_dict)
    try:
        response = df_data.to_sql(table_name, driver, index=False, if_exists='append')
        print(f"Se han insertado {response} nuevos registros")
    except Exception as ex:
        print(ex)

In [10]:
def insertData2SQL_categoria(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        for index, row in df_data.iterrows():
            estado_value = "'1'" if row.to_dict()['estado'] else "'0'"
            query = f"INSERT INTO {table_name} VALUES ({row.to_dict()['idcategoria']}, '{row.to_dict()['nombre']}', '{row.to_dict()['descripcion']}', CAST({estado_value} AS BIT))"
            cursor.execute(query)
        db_pg_conn.commit()
        print(f"Se han insertado {len(df_data)} nuevos registros")
    except Exception as ex:
        print(ex)

In [11]:
def insertData2SQL_rol(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        for index, row in df_data.iterrows():
            estado_value = "'1'" if row.to_dict()['estado'] else "'0'"
            query = f"INSERT INTO {table_name} VALUES ({row.to_dict()['idrol']}, '{row.to_dict()['nombre']}', '{row.to_dict()['descripcion']}', CAST({estado_value} AS BIT))"
            cursor.execute(query)
        db_pg_conn.commit()
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}")
    except Exception as ex:
        print(ex)

In [12]:
def insertData2SQL_articulo(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        for index, row in df_data.iterrows():
            estado_value = "'1'" if row.to_dict()['estado'] else "'0'"
            query = f"INSERT INTO {table_name} VALUES ({row.to_dict()['idarticulo']}, '{row.to_dict()['idcategoria']}', '{row.to_dict()['codigo']}', '{row.to_dict()['nombre']}', '{row.to_dict()['precio_venta']}', '{row.to_dict()['stock']}', '{row.to_dict()['descripcion']}', '{row.to_dict()['imagen']}', CAST({estado_value} AS BIT))"
            cursor.execute(query)
        db_pg_conn.commit()
        print(f"Se han insertado {len(df_data)} nuevos registros en la tabla {table_name}")
    except Exception as ex:
        print(ex)

In [13]:
driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOSTNAME}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

# Agregando datos a la tabla categoria

In [14]:
data_categoria = [
    {'idcategoria': 39582, 'nombre': 'Ropa y Accesorios', 'descripcion': 'Incluye prendas de vestir para hombres, mujeres y ninos, asi como accesorios como bolsos, sombreros, bufandas, etc.', 'estado': True},
    {'idcategoria': 82016, 'nombre': 'Calzado', 'descripcion': 'Incluye zapatos formales, tenis, botas, sandalias, para hombres, mujeres y ninos.', 'estado': True},
    {'idcategoria': 51470, 'nombre': 'Juguetes y Juegos', 'descripcion': 'Una amplia gama de juguetes para ninos de todas las edades, asi como juegos de mesa y articulos deportivos.', 'estado': True},
    {'idcategoria': 75369, 'nombre': 'Electronicos', 'descripcion': 'Desde telefonos moviles, computadoras portatiles y tabletas hasta accesorios como auriculares, cargadores y fundas.', 'estado': True},
    {'idcategoria': 20934, 'nombre': 'Mascotas', 'descripcion': 'Ofrece productos para el cuidado de mascotas, alimentos, juguetes y accesorios para perros, gatos y otras mascotas.', 'estado': True},
    {'idcategoria': 64218, 'nombre': 'Alimentos y Bebidas', 'descripcion': 'Desde alimentos enlatados y productos secos hasta bebidas como vinos, cervezas artesanales y licores.', 'estado': True},
    {'idcategoria': 13750, 'nombre': 'Libros y Papeleria', 'descripcion': 'Ofrece una selección de libros para todas las edades y generos, asi como suministros de papeleria como cuadernos, bolígrafos y lapices.', 'estado': True},
    {'idcategoria': 40879, 'nombre': 'Belleza y Cuidado Personal', 'descripcion': 'Incluye productos de cuidado de la piel, maquillaje, perfumes, asi como herramientas y accesorios para el cuidado del cabello.', 'estado': True},
    {'idcategoria': 97521, 'nombre': 'Joyeria', 'descripcion': 'Desde anillos, pulseras y collares hasta relojes y joyeria personalizada.', 'estado': True},
    {'idcategoria': 32684, 'nombre': 'Hogar y Decoracion', 'descripcion': 'Ofrece muebles para el hogar, articulos de decoracion como cuadros y jarrones, asi como utensilios de cocina y electrodomesticos.', 'estado': True}
]

# Insertamos data en la tabla categoria
insertData2SQL_categoria(data_categoria, 'categoria', driver)

Se han insertado 10 nuevos registros


# Agregando datos a la tabla rol


In [15]:
data_roles = [
    {'idrol': 10001, 'nombre': 'Administrador', 'descripcion': 'Este rol tiene acceso completo al sistema y puede realizar todas las acciones.', 'estado': True},
    {'idrol': 10002, 'nombre': 'Moderador', 'descripcion': 'Este rol tiene permisos para moderar contenido y realizar acciones específicas.', 'estado': True},
    {'idrol': 10003, 'nombre': 'Usuario Estándar', 'descripcion': 'Este rol tiene acceso limitado y puede realizar acciones básicas en el sistema.', 'estado': True},
    {'idrol': 10004, 'nombre': 'Soporte Técnico', 'descripcion': 'Este rol proporciona soporte técnico y tiene acceso a herramientas de diagnóstico.', 'estado': True},
    {'idrol': 10005, 'nombre': 'Invitado', 'descripcion': 'Este rol tiene acceso limitado y solo puede ver contenido público.', 'estado': True}
]

# Insertamos data en la tabla rol
insertData2SQL_rol(data_roles, 'rol', driver)

Se han insertado 5 nuevos registros en la tabla rol


# Agregando datos a la tabla articulo

In [16]:
from faker import Faker
import random

fake = Faker()

# Lista de palabras clave que representan tipos comunes de artículos
palabras_clave = ["Camisa", "Pantalón", "Zapatos", "Tenis", "Vestido", "Reloj", "Teléfono", "Portátil", "Tableta", "Juguete", "Libro", "Silla", "Mesa", "Lámpara", "Cuadro", "Cocina", "Cama", "Alfombra", "Jarrón", "Perfume", "Maquillaje", "Galleta", "Pelota", "Anillo", "Suéter", "Bolígrafo"]

nombres_articulos = []

# Generar nombres de artículos aleatorios combinando palabras aleatorias con palabras clave
for _ in range(1000):
    palabra_aleatoria = fake.word()
    nombre_articulo = random.choice(palabras_clave) + " " + palabra_aleatoria.capitalize()
    nombres_articulos.append(nombre_articulo)

print(nombres_articulos[:100])  # Imprimir los primeros 10 nombres de artículos generados


['Jarrón Guess', 'Suéter Through', 'Vestido Employee', 'Tenis Career', 'Camisa Among', 'Pantalón Somebody', 'Bolígrafo Major', 'Perfume High', 'Maquillaje On', 'Perfume Decade', 'Cuadro Then', 'Galleta Alone', 'Camisa Remain', 'Lámpara Most', 'Tenis Know', 'Maquillaje Information', 'Vestido Hour', 'Lámpara Degree', 'Tenis Many', 'Tableta Fast', 'Pelota Mrs', 'Tableta Meet', 'Anillo Bring', 'Libro Visit', 'Reloj Professional', 'Pelota Understand', 'Lámpara Partner', 'Lámpara Begin', 'Perfume Dinner', 'Tenis Race', 'Lámpara Risk', 'Perfume Among', 'Libro Federal', 'Tableta System', 'Cocina Sister', 'Libro Side', 'Teléfono Mission', 'Alfombra Evidence', 'Pelota Hit', 'Cuadro Task', 'Portátil Group', 'Reloj Build', 'Portátil Do', 'Juguete Cost', 'Maquillaje He', 'Silla Product', 'Silla Good', 'Tenis Talk', 'Tableta Practice', 'Libro Ready', 'Perfume Certain', 'Camisa Season', 'Juguete Husband', 'Silla Art', 'Perfume Mrs', 'Silla Tax', 'Perfume Sense', 'Pelota Middle', 'Alfombra Fall', 'Lám

In [17]:
def generar_codigo(length):
    caracteres = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    codigo = ''.join(random.choice(caracteres) for _ in range(length))
    return codigo

In [18]:
len(nombres_articulos)

1000

In [19]:
cantidad_articulos = 1000

data_articulos = []

for index, articulo in enumerate(range(cantidad_articulos)):
    nuevo_articulo = {
        'idarticulo': index,
        'idcategoria': random.sample(data_categoria, 1)[0]['idcategoria'],
        'codigo': generar_codigo(8),
        'nombre': nombres_articulos[index],
        'precio_venta': round(random.uniform(10, 1000),2),
        'stock': np.random.randint (1, 100),
        'descripcion': f'Artículo {nombres_articulos[index]} en venta en tienda',
        'imagen': 'fig_' + str(index),
        'estado': True
    }
    data_articulos.append(nuevo_articulo)
insertData2SQL_articulo(data_articulos, 'articulo', driver)

Se han insertado 1000 nuevos registros en la tabla articulo


In [14]:
# Consulta para obtener el nombre de todas las tablas en el esquema público
consulta_tablas = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public' 
AND table_type = 'BASE TABLE';
"""
# Ejecutar la consulta
cursor.execute(consulta_tablas)
# Obtener los nombres de las tablas
tablas = cursor.fetchall()

In [15]:
# Iterar sobre las tablas y eliminarlas una por una
for tabla in tablas:
    eliminar_tabla = f"DROP TABLE IF EXISTS {tabla[0]} CASCADE;"
    cursor.execute(eliminar_tabla)

# Confirmar los cambios y cerrar la conexión
db_pg_conn.commit()
db_pg_conn.close()